In [1]:
!source activate twitterCOVID

In [7]:
from tweepy import OAuthHandler, AppAuthHandler
from tweepy import API, TweepError
######### Importing with relative path
import importlib.util
spec = importlib.util.spec_from_file_location("secrets", "../secrets.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
#from ../secrets import *
##########################################
from textblob import TextBlob
import os
import json
import dataset
from datafreeze import freeze
import pandas as pd

In [8]:

# Consumer key authentication
auth = AppAuthHandler(foo.consumer_key, foo.consumer_secret) #using appauthhandler to retreive at a faster rate

# Access key authentication
# auth.set_access_token(foo.access_token, foo.access_token_secret)

# Set up the API with the authentication handler
api = API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify = True)

## Problem: SearchAPI is limited in many ways (max 100 tweets per query is just one of them)

In [9]:
search = api.search(q = "covid", until = "2020-04-02", count = 100, tweet_mode = "extended")

In [10]:
jsonres = [resi._json for resi in search]

In [11]:
results = pd.DataFrame(jsonres)

In [12]:
results.head()

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,retweet_count,favorite_count,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status
0,Wed Apr 01 23:59:59 +0000 2020,1245501195554369536,1245501195554369536,RT @DonaldJTrumpJr: The same media wants to st...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,18781,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Wed Apr 01 23:59:59 +0000 2020,1245501195373948929,1245501195373948929,RT @Econ_Marshall: Whatever happened to the Se...,False,"[0, 81]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,55,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Wed Apr 01 23:59:59 +0000 2020,1245501195327848449,1245501195327848449,"If I had a dollar..."" COVID edition: If I had...",False,"[0, 281]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,12,78,False,False,en,NaN,NaN,NaN,NaN,NaN
3,Wed Apr 01 23:59:59 +0000 2020,1245501195315220480,1245501195315220480,@look_now Trotzdem können sie die Krankheit ve...,False,"[10, 149]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'de', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.245501e+18,...,0,0,False,False,de,NaN,NaN,NaN,NaN,NaN
4,Wed Apr 01 23:59:59 +0000 2020,1245501195273285632,1245501195273285632,RT @RexChapman: Sandra the orangutang started ...,False,"[0, 139]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,...,105067,0,False,False,en,NaN,NaN,NaN,NaN,NaN


## Cracking Twitter APIs

From [link](https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./). The seacrh method has some `max_id` and `since_id` arguments we can exploit to our advantage. These are like upper and lower boundaries for our tweets, and given tweets come sorted by post time, we can retrieve many many tweets going back up to 1 week.

In [ ]:
import sys
import jsonpickle
import os

searchQuery = "covid"
maxTweets = 1000
tweetsPerQry = 100 # max set by twitter api
fName = "LastWeek.csv"

# the api search output SearchObject is an object that can be turned into a list of json str with the _json method
# the search object itself can be indexed like a list
sinceID = None # no lower limit (as far as necessary)
maxID = -1 #exploit negative indexing, last tweet
tweetCount = 0
# the ID is just an ordered number, If I tweet now and my tweet ID is 1 and
# your tweet comes next your tweet id will be 2


print("Downloading max {0} tweets".format(maxTweets))

with open(fName, 'w') as f:
    while tweetCount < maxTweets:
        try:
            if (maxID<=0):
                # if your maxID is negative (1st iteration) do this
                if (not sinceID):
                    new_tweets = api.search(q=searchQuery, count = tweetsPerQry)
                else: # if sinceID exists set sinceID as that minimum
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            since_id=sinceID)
            else: # if maxID is positive ? 
                if (not sinceID): # if sinceID not defined don't set low boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1))
                else: # if sinceID exist set low and up boundary
                    new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                            max_id=str(maxID - 1),
                                            since_id=sinceID)
            if not new_tweets:
                print("No more tweets found")
                break
            for tweet in new_tweets:
                f.write(jsonpickle.encode(tweet._json))
            tweetCount += len(new_tweets)
            print("Downloaded {0} tweets".format(tweetCount))
            maxID = new_tweets[-1].id # set maxID to the latest ID in the previous search
        except TweepError as e:
            # Just exit if any error
            print("some error : " + str(e))
            break
print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

In [ ]:
import pandas as pd
import jsonpickle
tweets = []

for line in open("BackSearch/SomeKeywords.json", "r"):
    tweets.append(json.loads(line))
tweets = pd.DataFrame(tweets)
tweets


In [ ]:
res = json.loads("BackSearch/SomeKeywords.json")

In [ ]:
import pandas as pd

In [ ]:
pd.read_json("BackSearch/SomeKeywords.json", lines = True)

In [ ]:
pdf = pd.read_csv("BackSearch/SomeKeywords.csv")

In [ ]:
jsonres[0].keys()